In [80]:
# imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
%matplotlib inline

### Dataset Loading

In [81]:
df = pd.read_excel("flight_price.xlsx")
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


### Feature Engineering

handling `Date_of_Journey` Attribute

In [83]:
df['Date'] = df['Date_of_Journey'].str.split('/').str[0].astype(int)
df['Month'] = df['Date_of_Journey'].str.split('/').str[1].astype(int)
df['Year'] = df['Date_of_Journey'].str.split('/').str[2].astype(int)

In [84]:
df.drop('Date_of_Journey',axis=1,inplace=True)

handling `Arrival_Time` Attribute

In [85]:
# Sampling few contents 
df['Arrival_Time'].unique()[:30]

array(['01:10 22 Mar', '13:15', '04:25 10 Jun', '23:30', '21:35', '11:25',
       '10:25 13 Mar', '05:05 02 Mar', '19:15', '23:00', '22:55', '13:55',
       '09:20 10 May', '19:50', '19:15 04 Mar', '12:35 13 Jun',
       '19:15 13 Jun', '12:35 28 May', '19:20', '13:20 19 Mar', '06:50',
       '21:50', '08:15 07 May', '12:35 10 Jun', '03:35 02 Jun', '12:55',
       '12:35', '09:20', '23:15', '12:35 16 Jun'], dtype=object)

In [86]:
# Removing the date from the time
df['Arrival_Time'] = df['Arrival_Time'].apply(
    lambda x: x.split(' ')[0]
)

In [87]:
df['Arrival_Time'].unique()[:30]

array(['01:10', '13:15', '04:25', '23:30', '21:35', '11:25', '10:25',
       '05:05', '19:15', '23:00', '22:55', '13:55', '09:20', '19:50',
       '12:35', '19:20', '13:20', '06:50', '21:50', '08:15', '03:35',
       '12:55', '23:15', '21:00', '20:20', '16:45', '19:00', '18:10',
       '15:15', '08:35'], dtype=object)

In [88]:
df['Arrival_hour'] = df['Arrival_Time'].str.split(':').str[0].astype(int)
df['Arrival_min'] = df['Arrival_Time'].str.split(':').str[1].astype(int)

In [89]:
df.drop('Arrival_Time',axis=1,inplace=True)

handling `Dep_Time` Attribute

In [90]:
df['Dep_Time'].unique()[:30]

array(['22:20', '05:50', '09:25', '18:05', '16:50', '09:00', '18:55',
       '08:00', '08:55', '11:25', '09:45', '20:20', '11:40', '21:10',
       '17:15', '16:40', '08:45', '14:00', '20:15', '16:00', '14:10',
       '22:00', '04:00', '21:25', '21:50', '07:00', '07:05', '09:50',
       '14:35', '10:35'], dtype=object)

In [91]:
df['Departure_hour'] = df['Dep_Time'].str.split(':').str[0].astype(int)
df['Departure_min'] = df['Dep_Time'].str.split(':').str[1].astype(int)

In [92]:
df.drop('Dep_Time',axis=1,inplace=True)

handling `Total_Stops` Attribute

In [93]:
df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [94]:
df['Total_Stops'].isnull().sum()

np.int64(1)

In [95]:
df[df['Total_Stops'].isnull()]

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Departure_hour,Departure_min
9039,Air India,Delhi,Cochin,NaN,23h 40m,NaN,No info,7480,6,5,2019,9,25,9,45


Only one value is missing, It might be missing at random. So replacing this categorical value using Mode value imputation. 

In [96]:
df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

Performing `Ordinal Encoding` and handling null values by `Mode Value Imputation`

In [97]:
df['Total_Stops'] = df['Total_Stops'].map({
    'non-stop': 0,
    '1 stop': 1,
    '2 stops': 2,
    '3 stops': 3,
    '4 stops': 4,
    np.nan: 1
})

In [98]:
# Checking for null 
df[df['Total_Stops'].isnull()]

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Departure_hour,Departure_min


In [99]:
df.head(3)

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Departure_hour,Departure_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,0,No info,3897,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2,No info,7662,1,5,2019,13,15,5,50
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2,No info,13882,9,6,2019,4,25,9,25


As `Source` and `Destination` are present, Route is a unnecessary attribute so dropping it.

In [100]:
df.drop('Route',axis=1,inplace=True)

Handling `Duration` Attribute

As the Duration attribute contains either only **h** or only **m** for few contents so extraction via regex and handling null is the best way to handle this particular attribute.

In [101]:
df[["Duration_hours", "Duration_minutes"]] = df["Duration"].str.extract(r'(\d+)h?\s*(\d+)m?').fillna(0).astype(int)

In [102]:
df['Duration_hours'].unique()

array([ 2,  7,  1,  5,  4, 15, 21, 25, 13, 12, 26, 22, 20, 10,  6, 11,  8,
       16, 19,  3,  0, 27, 14,  9, 18, 17, 30, 23, 24, 28, 29, 37, 34, 38,
       35, 36, 33, 32, 31, 42, 39, 41, 40, 47])

In [103]:
df['Departure_min'].unique()

array([20, 50, 25,  5,  0, 55, 45, 40, 10, 15, 35, 30])

In [104]:
df['Total_mins'] = df['Duration_hours'] * 60 + df['Duration_minutes']
df['Total_mins'].unique()[:30]

array([ 170,  445,   69,  325,  285,  145,  930, 1265, 1530,  470,  795,
        155,  135,  730, 1595,  270, 1355,  123, 1235,  310,  920,  175,
        800,  910,  345,  355,  805,  122,  330,  625])

In [105]:
df.drop('Duration',axis=1,inplace=True)

In [106]:
df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Departure_hour,Departure_min,Duration_hours,Duration_minutes,Total_mins
0,IndiGo,Banglore,New Delhi,0,No info,3897,24,3,2019,1,10,22,20,2,50,170
1,Air India,Kolkata,Banglore,2,No info,7662,1,5,2019,13,15,5,50,7,25,445
2,Jet Airways,Delhi,Cochin,2,No info,13882,9,6,2019,4,25,9,25,1,9,69
3,IndiGo,Kolkata,Banglore,1,No info,6218,12,5,2019,23,30,18,5,5,25,325
4,IndiGo,Banglore,New Delhi,1,No info,13302,1,3,2019,21,35,16,50,4,45,285


In [112]:
df.Price.isnull().sum()

np.int64(0)

In [107]:
df.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [108]:
df.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [109]:
df.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

In [110]:
df.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Airline           10683 non-null  object
 1   Source            10683 non-null  object
 2   Destination       10683 non-null  object
 3   Total_Stops       10683 non-null  int64 
 4   Additional_Info   10683 non-null  object
 5   Price             10683 non-null  int64 
 6   Date              10683 non-null  int64 
 7   Month             10683 non-null  int64 
 8   Year              10683 non-null  int64 
 9   Arrival_hour      10683 non-null  int64 
 10  Arrival_min       10683 non-null  int64 
 11  Departure_hour    10683 non-null  int64 
 12  Departure_min     10683 non-null  int64 
 13  Duration_hours    10683 non-null  int64 
 14  Duration_minutes  10683 non-null  int64 
 15  Total_mins        10683 non-null  int64 
dtypes: int64(12), object(4)
memory usage: 1.3+ MB


This concludes the EDA for Flight Price Dataset. The `Airline, Source, Destination, Additional_Info` columns can be `OnehotEncoded` or `Embedded` based on the need.